In [ ]:
!pip install langchain --quiet
!pip install langchain-openai --quiet
!pip install langchain-community --quiet
!pip install streamlit --quiet
!pip install PyMuPDF --quiet
!pip install chromadb --quiet
!pip install pyngrok --quiet
pip install pypdf --quiet
!pip install langchain-community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.1/571.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requiremen

In [ ]:
#pip install langchain_chroma --quiet
!pip install langgraph --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')
os.environ['NGROK_AUTH_TOKEN'] = userdata.get('NGROK_AUTH_TOKEN')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [ ]:
%%writefile app.py
import os
import tempfile
import streamlit as st
from operator import itemgetter
import pandas as pd

# ----------------------------------
# Page configuration
# ----------------------------------
st.set_page_config(page_title="File QA Chatbot", page_icon="🤖")
st.title("Let's Chat With Your Files 🤖")

# ----------------------------------
# API Keys setup (via st.secrets or environment)
# ----------------------------------
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_API_KEY')


# ----------------------------------
# Original RAG Pipeline Code (preserved)
# ----------------------------------
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import END, StateGraph
from typing import List
from typing_extensions import TypedDict

# Initialize embedding model
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

# Sidebar: Multiple PDF upload
st.sidebar.header("Upload PDFs")
uploaded_files = st.sidebar.file_uploader(
    "Upload one or more PDF files", type="pdf", accept_multiple_files=True
)

pdf_paths = []
if uploaded_files:
    st.sidebar.write(f"Uploading {len(uploaded_files)} file(s)...")
    for file in uploaded_files:
        tfile = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
        tfile.write(file.getvalue())
        pdf_paths.append(tfile.name)

if not pdf_paths:
    st.info("Please upload PDF files in the sidebar to begin.")
    st.stop()

# Load PDFs and split into documents
all_docs = []
for path in pdf_paths:
    loader = PyPDFLoader(path)
    pdf_docs = loader.load()  # Each page is a separate Document
    all_docs.extend(pdf_docs)
#st.write(f"Loaded **{len(all_docs)}** total pages from PDFs.")

# Chunk the documents (~1000 characters with 100 overlap)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)
chunked_docs = text_splitter.split_documents(all_docs)
#st.write(f"Total chunks after splitting: **{len(chunked_docs)}**")

# Create vector DB of docs and embeddings
persist_directory = "./papers_db"
chroma_db = Chroma.from_documents(
    documents=chunked_docs,
    collection_name='papers',
    embedding=openai_embed_model,
    collection_metadata={"hnsw:space": "cosine"},
    persist_directory=persist_directory
)

similarity_threshold_retriever = chroma_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.3}
)

# Define grading LLM and prompt for document relevance
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

llm_grader = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_grader = llm_grader.with_structured_output(GradeDocuments)

SYS_PROMPT = (
    "You are an expert grader assessing relevance of a retrieved document to a user question.\n"
    "Follow these instructions for grading:\n"
    "  - If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.\n"
    "  - Your grade should be either 'yes' or 'no' to indicate whether the document is relevant to the question or not."
)
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", """Retrieved document:
{document}
User question:
{question}
"""),
    ]
)
doc_grader = grade_prompt | structured_llm_grader

# Create QA generation chain components
qa_prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If no context is present or if you don't know the answer, just say that you don't know the answer.
Do not make up the answer unless it is there in the provided context.
Give the answer to the point with regard to the question no more than three lines.
Question:
{question}
Context:
{context}
Answer:
"""
prompt_template = ChatPromptTemplate.from_template(qa_prompt_template)
chatgpt = ChatOpenAI(model_name='gpt-4o', temperature=0)

def format_docs(docs: List[Document]) -> str:
    if hasattr(docs[0], 'content'):
        return "\n\n".join(doc.content for doc in docs)
    elif hasattr(docs[0], 'page_content'):
        return "\n\n".join(doc.page_content for doc in docs)
    return ""

qa_rag_chain = (
    {
        "context": (itemgetter('context') | RunnableLambda(format_docs)),
        "question": itemgetter('question')
    }
    | prompt_template
    | chatgpt
    | StrOutputParser()
)

# Define the agentic RAG pipeline graph using StateGraph
class GraphState(TypedDict):
    question: str
    generation: str
    web_search_needed: str
    documents: List[Document]

def retrieve(state):
    st.write("**[Retrieval from Vector DB]**")
    question = state["question"]
    documents = similarity_threshold_retriever.invoke(question)
    return {"documents": documents, "question": question}

def grade_documents(state):
    st.write("**[Checking Document Relevance]**")
    question = state["question"]
    documents = state["documents"]
    filtered_docs = []
    web_search_needed = "No"
    if documents:
        for d in documents:
            score = doc_grader.invoke({"question": question, "document": d.page_content})
            if score.binary_score.lower() == "yes":
                st.write("Document graded: **Relevant**")
                filtered_docs.append(d)
            else:
                st.write("Document graded: **Not Relevant**")
                web_search_needed = "Yes"
        # End for
    else:
        st.write("No documents retrieved.")
        web_search_needed = "Yes"
    return {"documents": filtered_docs, "question": question, "web_search_needed": web_search_needed}

def rewrite_query(state):
    st.write("**[Rewriting Query]**")
    # Here you could add a rewriting chain. For now, we simply pass through.
    question = state["question"]
    return {"documents": state["documents"], "question": question}

def web_search(state):
    from openai import OpenAI
    client = OpenAI()
    st.write("**[Performing Web Search]**")
    question = state["question"]
    documents = state["documents"]
    completion = client.chat.completions.create(
        model="gpt-4o-search-preview",
        web_search_options={"search_context_size": "low"},
        messages=[{"role": "user", "content": question}]
    )
    web_results = completion.choices[0].message
    # Append the web result to documents (assumes it has a page_content attribute)
    documents.append(web_results)
    return {"documents": documents, "question": question}

def generate_answer(state):
    st.write("**[Generating Answer]**")
    question = state["question"]
    documents = state["documents"]
    generation = qa_rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def decide_to_generate(state):
    st.write("**[Deciding Next Step]**")
    if state["web_search_needed"] == "Yes":
        st.write("Rewriting query due to non-relevant documents.")
        return "rewrite_query"
    else:
        st.write("Proceeding to generate answer.")
        return "generate_answer"

agentic_rag = StateGraph(GraphState)
agentic_rag.add_node("retrieve", retrieve)
agentic_rag.add_node("grade_documents", grade_documents)
agentic_rag.add_node("rewrite_query", rewrite_query)
agentic_rag.add_node("web_search", web_search)
agentic_rag.add_node("generate_answer", generate_answer)
agentic_rag.set_entry_point("retrieve")
agentic_rag.add_edge("retrieve", "grade_documents")
agentic_rag.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {"rewrite_query": "rewrite_query", "generate_answer": "generate_answer"},
)
agentic_rag.add_edge("rewrite_query", "web_search")
agentic_rag.add_edge("web_search", "generate_answer")
agentic_rag.add_edge("generate_answer", END)
agentic_rag = agentic_rag.compile()

# ----------------------------------
# Chatbot UI using StreamlitChatMessageHistory
# ----------------------------------
from langchain_community.chat_message_histories import StreamlitChatMessageHistory

# Initialize conversation history using StreamlitChatMessageHistory
streamlit_msg_history = StreamlitChatMessageHistory(key="langchain_messages")

# Shows the first message when the app starts
if len(streamlit_msg_history.messages) == 0:
    streamlit_msg_history.add_ai_message("What would you like to ask to your documents?")

# Render existing conversation messages
for msg in streamlit_msg_history.messages:
    st.chat_message(msg.type).write(msg.content)

# Helper function to compile chat history into a string (if needed)
def compile_chat_history(messages):
    history = ""
    for msg in messages:
        role = msg.type.capitalize()
        history += f"{role}: {msg.content}\n"
    return history

# Chat input and processing
if user_prompt := st.chat_input("Enter your question:"):
    # Display user message and add it to message history
    st.chat_message("user").write(user_prompt)
    streamlit_msg_history.add_user_message(user_prompt)

    # Optionally compile chat history (if you want to include it in the prompt)
    chat_history_text = compile_chat_history(streamlit_msg_history.messages)

    # Invoke the agentic RAG pipeline with the current question
    state = {"question": user_prompt}
    result = agentic_rag.invoke(state)
    answer = result.get("generation", "Sorry, I could not generate an answer.")

    # Display bot response and update conversation history
    st.chat_message("assistant").write(answer)
    streamlit_msg_history.add_ai_message(answer)


Overwriting app.py


In [ ]:
!streamlit run app.py --server.port=8989 &>./logs.txt &

In [ ]:
from pyngrok import ngrok
import yaml
import os
# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
# with open('ngrok_credentials.yml', 'r') as file:
#     NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(os.environ['NGROK_AUTH_TOKEN'] )

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://1679-35-230-16-184.ngrok-free.app
